In [5]:
import wikichatter as wc
import mwapi
import requests
import json
import csv

In [6]:
#When working with mwapi, you should begin your file with a session, with a user_agent
#this allows you to make multiple calls without getting shut out of the api

session = mwapi.Session('https://en.wikipedia.org', user_agent='ewhit')

#Creating a query
>params holds the parameters you want to use when calling mwapi. This makes it easy to  
manipulate each parameter, particularly "titles" Here, we can see that titles  
has been set to ["Wikipedia:List of controversial issues"](https://en.wikipedia.org/wiki/Wikipedia:List_of_controversial_issues) 
These parameters will do the following:
    >>Tell mwapi to make a query (action), to get all of the links present (prop) on the page 
    >>"Wikipedia:List of controversial issues" (titles), and to return this in json format (format)
>The query will end up looking like this:  
    >>[https://en.wikipedia.org/w/api.php?action=query&prop=links&titles=Wikipedia:List%20of%20controversial%20issues&pllimit=max](https://en.wikipedia.org/w/api.php?action=query&prop=links&titles=Wikipedia:List%20of%20controversial%20issues&pllimit=max)

In [7]:
def params():
    return{
    "action": "query",
    "titles" : "Wikipedia:List of controversial issues",
    "prop": "links",
    "format": "json",
    "pllimit" : "1"
    }


#kwargs calls params in order to be set - this is what you will end up passing
kwargs=params()

#This holds the results of your query, and continuation=True tells session.get to keep going when it reaches the
#limit (of 50, I believe), so that you can get *all* of the links on the page
#Because we wanted all the titles on the page, query will be a generator
query = session.get(**kwargs, continuation = True)

In [8]:
#Pagelist is a list that will ultimately hold the titles of all the controversial pages
pageList = []

#This loops through all the results of the earlier query
#However, because what we want to do next is acquire the talk page for each controversial
#page, we will need to do a bit of trimming
#Ultimately, this bit of code will put "Talk:page title" into our list of pages to visit later
for request in query:
    title = json.dumps(request)
    index = title.find('links')
    temp = title[index:]
    index2 = temp.find('title')
    temp2 = temp[index2:]
    temp3 = temp2[9:]
    index3 = temp3.find('"')
    temp4 = temp3[:index3]
    toTalk = "Talk:"
    temp5 = toTalk + temp4
    pageList.append(temp5)
    

print("end for loop")

end for loop


#Queries
> Each query will return something like this: 
>> {"query": {"pages": {"27985631": {"links": [{"ns": 0, "title": "1919 World Series"}], "ns": 4, "title": "Wikipedia:List of controversial issues", "pageid": 27985631}}}, "continue": {"plcontinue": "27985631|0|1953_Iranian_coup_d'\u00e9tat", "continue": "||"}}  

>Because we want to query each individual talk page, we will need to parse the title of the talk page from what each query returns, which is what the above code does. After this loop has run, we should have a list of things that look like "Talk:1919 World Series" 

>After parsing and setting the param to the appropriate parameters, a query that looks like this should be generated:  
>>[https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&titles=Talk:1919%20World%20Series&format=json](https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&titles=Talk:1919%20World%20Series&format=json)  
>This will get the content of the page queried  
>This content is then passed to wc.parse so that it can be parsed - it looks quite unreadable initially  
>This appears to be where errors are happening

In [ ]:
#This is a dictionary to hold the talk page title, and the content of that talk page
content = {}

#This loops through each page in our list of controversial talk pages
for page in pageList:
    
    #Once again, we want to define the parameters we will send to session.get
    #Here, we are again querying, and "titles" : page will query the page with the corresponding title
    #(which is why we just stored the talk page titles earlier)
    #"prop" : "revisions" gets the most recent revision, and "rvprop" : "content" gets the current content
    def param(page):
        return{
        "action": "query",
        "titles" : page,
        "prop" : "revisions",
        "rvprop" : "content",
        "format": "json",
        }
    
    #Here we again set kwargs to send to session.get
    kwargs2 = param(page)
    query = session.get(**kwargs2, continuation=True )
    
    #Because this portion of code was throwing errors, I have used several try-catch blocks
        #Several were used because the first kind returned a TypeError, so I caught that as well
    #try-catch block
    i = 0
    for request in query:
        #This is the text that results from the individual query (so, the talk page content)
        text = json.dumps(request)

        try:
            #Here, we try to parse the talk page content
            try:
                parsed_text = wc.parse(text)
            except NameError:
                i = i + 1  
            except NoUsernameError(parsed_text):
                i = i + 1
        except TypeError:
            i = i + 1

        #Here, we add an entry to the dictionary of the current page's title and parsed text
        content[page] = parsed_text
            
print("finished querrying")

In [ ]:
#This simply prints the page titles and their corresponding cleaned text to a file
#with open("parsedTalkPages.csv", "w", newline = '') as w:
    #writer = csv.writer(w, delimiter = ',')
    #for key, value in content.items():
     #   writer.writerow([key, value])

#Output  
>The following is a line from the csv that this code generates:  

Talk:Incest	 

{'sections': [{'comments': [{'text_blocks': ['{"batchcomplete": "", "query": {"pages": {"23716981": {"pageid": 23716981, "title": "Talk:Incest", "revisions": [{"contentmodel": "wikitext", "contentformat": "text/x-wiki", "*": "{{Talk header|search=yes}}\\n{{User:MiszaBot/config\\n|archiveheader = {{aan}}\\n|maxarchivesize = 200K\\n|counter = 3\\n|minthreadsleft = 7\\n|algo = old(30d)\\n|archive = Talk:Incest/Archive %(counter)d\\n}}\\n{{Not a forum|incest}}\\n{{WikiProjectBannerShell|1=\\n{{WikiProject Psychology|class=Start|importance=High}}\\n{{WikiProject Sexuality|class=start|importance=high}}\\n{{WikiProject Anthropology|class=start|importance=high}}\\n{{Vital article|class=Start|topic=Life|level=4}}\\n}}\\n\\n== Clarification Needed ==\\n\\nAfter the biblical diagram, there\'s this line that remains without proper citation or explanation \\"Apart from the questionable case of the daughter [clarification needed]\\"  <small class=\\"autosigned\\">\\u2014&nbsp;Preceding [[Wikipedia:Signatures|unsigned]] comment added by [[Special:Contributions/189.75.148.65|189.75.148.65]] ([[User talk:189.75.148.65|talk]]) 23:09, 30 March 2015 (UTC)</small><!-- Template:Unsigned IP --> <!--Autosigned by SineBot-->\\n\\nI want to know that what leads the Biological Parents to commit to incest. [[User:Saket4353|Saket4353]] ([[User talk:Saket4353|talk]]) 03:43, 28 April 2016 (UTC)\\n\\n== \'\'Incestualist\'\' and \'\'Incestophile\'\' ==\\n\\nMy recent revert was cut off due to some kind of technical problem so the edit summary is probably odd.  What I was trying to say is that the sources for \'\'Incestualist\'\' and \'\'Incestophile\'\' appear to not be very strong.  They do not provide any proof these are common, accepted terms. Furthermore,, Wible deliberately uses it in a \'\'sarcastic\'\' way in making an argument for acceptance of homosexuality.[[User:Legitimus|Legitimus]] ([[User talk:Legitimus|talk]]) 20:31, 29 July 2015 (UTC)\\n:: Are those terms correctly formed? I guess we could include incestophobia as well. That would be someone that is opposed to incest. --[[Special:Contributions/41.146.63.190|41.146.63.190]] ([[User talk:41.146.63.190|talk]]) 12:55, 6 October 2016 (UTC)\\n:::I can\'t really say if they\'re are correctly formed as I lack the needed knowledge of English linguistics, though I have noticed that the internet has created a variety of terribly mangled terms as well as corruption of existing words true meanings.   However, most important is that these all appear to be made-up terms used by a handful of authors in a joking sort of way.  I could not find any good scientific or reliable \\"serious\\" usage of any of them, including the one you suggested.[[User:Legitimus|Legitimus]] ([[User talk:Legitimus|talk]]) 14:11, 6 October 2016 (UTC)\\n\\n== Weasel words ==\\n\\nThis article has some weasel words that need adjusting. \\n\\nPlease weigh in.  <small><span class=\\"autosigned\\">\\u2014&nbsp;Preceding [[Wikipedia:Signatures|unsigned]] comment added by [[User:CircularReason|CircularReason]] ([[User talk:CircularReason|talk]] \\u2022 [[Special:Contributions/CircularReason|contribs]]) 03:03, 12 September 2015 (UTC)</span></small><!-- Template:Unsigned --> <!--Autosigned by SineBot-->\\n\\n== External links modified ==\\n\\nHello fellow Wikipedians,\\n\\nI have just modified 5 external links on [[Incest]]. Please take a moment to review [https://en.wikipedia.org/w/index.php?diff=prev&oldid=783778764 my edit]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit [[User:Cyberpower678/FaQs#InternetArchiveBot|this simple FaQ]] for additional information. I made the following changes:\\n*Added archive https://web.archive.org/web/20131217044158/http://www.caselaw.nsw.gov.au/action/PJUDG?jgmtid=167373 to http://www.caselaw.nsw.gov.au/action/PJUDG?jgmtid=167373\\n*Added archive https://web.archive.org/web/20080904084118/http://mdn.mainichi.jp/20080720/0720_05.html to http://mdn.mainichi.jp/20080720/0720_05.html\\n*Added archive https://web.archive.org/web/20080904084448/http://mdn.mainichi.jp/20080720/0628.html to http://mdn.mainichi.jp/20080720/0628.html\\n*Added archive https://web.archive.org/web/20080903182854/http://mdn.mainichi.jp/20080720/ to http://mdn.mainichi.jp/20080720/\\n*Added archive https://web.archive.org/web/20081228061401/http://www.psychologytoday.com/articles/index.php?term=19930101-000023&page=1 to http://psychologytoday.com/articles/index.php?term=19930101-000023&page=1\\n*Added {{tlx|dead link}} tag to http://www.palgraveconnect.com/pc/doifinder/10.1057/9781137476159.0009\\n\\nWhen you have finished reviewing my changes, you may follow the instructions on the template below to fix any issues with the URLs.\\n\\n{{sourcecheck|checked=false|needhelp=}}\\n\\nCheers.\\u2014[[User:InternetArchiveBot|\'\'\'<span style=\\"color:darkgrey;font-family:monospace\\">InternetArchiveBot</span>\'\'\']] <span style=\\"color:green;font-family:Rockwell\\">([[User talk:InternetArchiveBot|Report bug]])</span> 16:07, 4 June 2017 (UTC)\\n\\n== External links modified ==\\n\\nHello fellow Wikipedians,\\n\\nI have just modified 2 external links on [[Incest]]. Please take a moment to review [https://en.wikipedia.org/w/index.php?diff=prev&oldid=792413901 my edit]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit [[User:Cyberpower678/FaQs#InternetArchiveBot|this simple FaQ]] for additional information. I made the following changes:\\n*Added archive https://web.archive.org/web/20090921025414/http://www.mlahanas.de/Greeks/Bios/Elpinice.html to http://www.mlahanas.de/Greeks/Bios/Elpinice.html\\n*Added archive https://web.archive.org/web/20101103062452/http://www.sanathanadharma.com/samskaras/marriage/mar3.htm to http://www.sanathanadharma.com/samskaras/marriage/mar3.htm\\n\\nWhen you have finished reviewing my changes, you may follow the instructions on the template below to fix any issues with the URLs.\\n\\n{{sourcecheck|checked=false|needhelp=}}\\n\\nCheers.\\u2014[[User:InternetArchiveBot|\'\'\'<span style=\\"color:darkgrey;font-family:monospace\\">InternetArchiveBot</span>\'\'\']] <span style=\\"color:green;font-family:Rockwell\\">([[User talk:InternetArchiveBot|Report bug]])</span> 11:13, 26 July 2017 (UTC)\\n\\n== Professor Steve Jones material and [[WP:Undue weight]] ==\\n\\nAs seen [https://en.wikipedia.org/w/index.php?title=Incest&diff=793175217&oldid=793089848 here] and [https://en.wikipedia.org/w/index.php?title=Incest&diff=793801740&oldid=793688719 here], I twice reverted content that [[User:Swingoswingo|Swingoswingo]] [https://en.wikipedia.org/w/index.php?title=Incest&diff=792813974&oldid=792688041 added]. With that second revert, you can see that Swingoswingo [https://en.wikipedia.org/w/index.php?title=Incest&diff=793688548&oldid=793584527 changed the material] so that it\'s not focused on the professor. For that second edit, I only reverted the \\"it is said to endanger the health of future generation\\" part. I reverted both times due to the fact that commentary from a lone professor in a section that is specifically about religious views is [[WP:Undue weight]]. Not only was the original content not speaking of any religious view, it was simply stating that it is common in the Muslim world for men to marry their nieces and cousins and that it endangers the health of future generations. Health with regard to incest is [https://en.wikipedia.org/w/index.php?title=Incest&oldid=793863930#Inbreeding already covered in the article], with better/more appropriate sources. It\'s also covered in the lead. Readers know that incest can lead to genetic problems. We don\'t need the inclusion of professor Steve Jones stating that it endangers the health of future generations. Furthermore, that health commentary is vague. \\n\\nThe [[User:Emir of Wikipedia|Emir of Wikipedia]] account [https://en.wikipedia.org/w/index.php?title=Incest&diff=793861653&oldid=793801740 re-added] Professor Steve Jones\'s name; I\'m sure he did this per [[WP:Intext]]. \\n\\nI object to all versions. Thoughts? This is something that may need to be taken to the [[WP:Neutral noticeboard]]. I state this noticeboard because WP:Undue weight is an aspect of WP:Neutral and there is no WP:Undue weight noticeboard. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 21:38, 4 August 2017 (UTC)\\n:I added the name as I believed that is was not a unanimously held opinion as it claims that Muslims marry nieces but above it says this is forbidden. Intext attribution should be used in this case. [[User:Emir of Wikipedia|Emir of Wikipedia]] ([[User talk:Emir of Wikipedia|talk]]) 22:30, 4 August 2017 (UTC)\\n\\n::In-text attribution is not what I object to; above, I was indicating that I understand why you added the name. But the in-text attribution in this case certainly signals WP:Undue weight. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 23:04, 4 August 2017 (UTC)\\n:::We don\'t need to go to any noticeboard. We are enough here to solve it.  [[User:Swingoswingo|Swingoswingo]] ([[User talk:Swingoswingo|talk]]) 14:09, 5 August 2017 (UTC)\\n:::If and when we get additional sources for the same claim, intext wont be necessary anymore :) [[User:Swingoswingo|Swingoswingo]] ([[User talk:Swingoswingo|talk]]) 14:14, 5 August 2017 (UTC)\\n:::What more I can think of is moving most of the Steve Jones words into the quote= parameter of the cite template. [[User:Swingoswingo|Swingoswingo]] ([[User talk:Swingoswingo|talk]]) 14:14, 5 August 2017 (UTC)\\n\\n::::We do need to go to a noticeboard since the content is disputed and the matter is not resolved in any way here. It\'s not like I will be agreeing with the addition. So I will now be taking the matter to the aforementioned noticeboard; hopefully, some editors will weigh in on the matter there or here. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 14:17, 6 August 2017 (UTC)\\n\\n:{{ping|Acroterion}} You reverted content being discussed here. [[User:Emir of Wikipedia|Emir of Wikipedia]] ([[User talk:Emir of Wikipedia|talk]]) 13:46, 7 August 2017 (UTC)\\n:I reverted an unexplained removal by abn IP of sourced material in an article prone to vandalism. If a consensus is established here for a sourced change, feel free to make the change. I have no opinion in the matter, I am not following this discussion. \'\'\'<font face=\\"Arial\\">[[User:Acroterion|<font color=\\"black\\">Acroterion</font>]] <small>[[User talk:Acroterion|<font color=\\"gray\\">(talk)</font>]]</small></font>\'\'\' 16:16, 7 August 2017 (UTC)\\n\\n:[https://en.wikipedia.org/w/index.php?title=Incest&diff=794338344&oldid=794328868 This] is Acroterion\'s revert. If no one weighs in at the noticeboard on the content, I\'ll simply start a [[WP:RfC]] on it. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 22:31, 7 August 2017 (UTC)\\nAlbeit cousin marriage is common in the Middle East as much more sources discuss in the article and in other pages, the claim that people marry their nieces is unfounded. There is no such claim except for his one guy. You can try to look in more sources like scientific journals or books. You won\'t find such claim. One hyperbolic claim by one guy who seems to have made it out of political inclination should not be seen as reliable. The source and the claim should be removed altogether. You can start a [[WP:RfC]] It\'s very easy to come to the conclusion that uncle to niece relationships are not just illegal but they are also not common in the middle east. [[User:CaliphoShah|CaliphoShah]] ([[User talk:CaliphoShah|talk]]) 07:18, 18 August 2017 (UTC)\\n\\n== External links modified ==\\n\\nHello fellow Wikipedians,\\n\\nI have just modified one external link on [[Incest]]. Please take a moment to review [[special:diff/809990777|my edit]]. If you have any questions, or need the bot to ignore the links, or the page altogether, please visit [[User:Cyberpower678/FaQs#InternetArchiveBot|this simple FaQ]] for additional information. I made the following changes:\\n*Added archive https://web.archive.org/web/20141101063712/http://socialwork.rutgers.edu/Libraries/VAWC/Trusiani_presentation.sflb.ashx to http://socialwork.rutgers.edu/Libraries/VAWC/Trusiani_presentation.sflb.ashx\\n\\nWhen you have finished reviewing my changes, you may follow the instructions on the template below to fix any issues with the URLs.\\n\\n{{sourcecheck|checked=false|needhelp=}}\\n\\nCheers.\\u2014[[User:InternetArchiveBot|\'\'\'<span style=\\"color:darkgrey;font-family:monospace\\">InternetArchiveBot</span>\'\'\']] <span style=\\"color:green;font-family:Rockwell\\">([[User talk:InternetArchiveBot|Report bug]])</span> 19:46, 12 November 2017 (UTC)\\n\\n== Between adults and children ==\\n\\nMy changes to this section were recently reverted by [[User:Legitimus]].  There were no comments made on either my talk page or the article talk page, so all I have to go on re what the issue with the change was the inline comment re the update.  The basic change to the article I am proposing is to note that when incest occurs between an adolescent son and a mother, it is sometimes the son is the abuser.  This may be a disturbing conclusion, as are all conclusions in this topic.  But it is supported in the literature, which is why I posted it.  The primary reference (Courtois) does in fact have an online preview available, contrary to what is stated in the inline comment.  That reference is available for preview at [https://books.google.com/books?isbn=0393313565] .  Look at pages 71-72 for this particular topic.  The secondary reference (Ward) is referred to by Courtois but may not have an online preview available.  Is it necessary that all references be available for free online?  There are certainly reliable sources that don\'t make all their content available for free.  But in this case material supporting the statement I made IS available in an online preview.  Also it isn\'t clear what the \\"variables\\" are that are unclear although based on another article that [[User:Legitimus]] reverted, this may refer to the age of the son.  I will try to find out the exact age of the sons referred to in this study, and also wait for a response to this message.  But absent a reason not to reinstate this edit, I will reinstate it in a few days after allowing time for a discussion.  I will focus primarily on this article, not the other article that was reverted, as I see this article as the primary focus of the change.  [[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 17:07, 27 November 2017 (UTC)\\n:I made a judgement call on the revert.  There is no need to act so offended by it.  Remember, reverting is not a final decision.  The combination of the claim being rather extraordinary and the text not explaining the issue completely suggested this might need some discussion first before including it.\\n:Regarding the sources, no it is not necessary for them to all be online, but this sensitive topic and the unusual claim necessitated there has to be some kind of way of verifying it.  When I clicked through wikipedia\'s link to google books, it came up empty for both books.  Your link solved only partially that problem.\\n:On the topic itself, there were things not made clear in the edit that are important.  First is whether violence (or the threat of, explicit or implied) or impairment were involved.  If the conscious mother went along with what happened without there being any threat or power imbalance against her, it sounds awfully suspect.  Second is the age of the male.  Obviously if the son is pre-pubescent, it casts serious doubt as to him being the \\"abuser\\" if the encounter is non-violent.  After all, pedophiles since the time they were first studied have made false or even [[Delusion|delusional]] claims of the child being the seducer.  Courtois states that the source \\"Arroyo, Eth, and Pynoos 1984\\" claim a case of an 8 year old, but also use the term \\"sexual assault,\\" implying there was some kind of aggression.  Page 72 of Courtois will not load for me so I can\'t make further commentary.  But these matters need to be made clear in the text.[[User:Legitimus|Legitimus]] ([[User talk:Legitimus|talk]]) 18:59, 27 November 2017 (UTC)\\n:: I\'m also skeptical of the claim involving the 8 year old.  Since it looks like you were able to access page 71, I\'m referring to the \\"adolescents\\" mentioned in the previous sentence.  In other words I\'m referring here to boys young enough to still legally be minors, but old enough to clearly be physically capable of initiating an attack.  Not 8 year olds.  But I would like to nail down the exact age we are talking about here.  I need to review the full original references which I haven\'t had a chance to do yet.[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 19:57, 27 November 2017 (UTC)\\n:::OK, I got page 72 to load and have a better picture now of the topic.  The reference to Ward seems to finally spell it out:  They\'re talking about *rape* by sons.  To explain, there is a subtle implication in the term \\"incest\\" that implies the encounter is non-violent or even consensual (i.e. between adults), but still \\"wrong.\\"  When violence enters the equation, it\'s rape/abuse, and the fact that it also happens to be incestuous is simply an aggravating factor.  This is probably why terms like \\"son-initiated incest\\" sound so strange.  A footnote on Page 236 referencing Ward supports this, claiming calling it incest \\"minimizes the reality of the violation.\\"  [[User:Legitimus|Legitimus]] ([[User talk:Legitimus|talk]]) 20:42, 27 November 2017 (UTC)\\n::::Based on the quote above from page 236, she seems to be arguing that what is generally described as father-daughter incest would be more accurately called rape.  After all, the title of her book is \\"Father Daughter Rape\\".  As such, she seems to be mainly focused on the father-daughter situation, but seems to touch on other forms of intrafamilial sexual violence as well, including the mother-son situation.  I want to be sure I\'m representing her views correctly before quoting her again.[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 07:20, 28 November 2017 (UTC)\\n:::::I support Legitimus\'s revert of you here and [https://en.wikipedia.org/w/index.php?title=Child_sexual_abuse&diff=prev&oldid=812307706 at the Child sexual abuse] article. Good to see you discussing this instead of simply reverting. Also good to see Legitimus working the matter out via looking over sources. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 03:39, 29 November 2017 (UTC)\\n::::::[[User:Flyer22 Reborn]]  I appreciate that and am currently working to research both references thoroughly before reinstating content.  At the same time there is clearly content in both references relevant to this article so I expect to reinstate some content-not necessarily exactly matching my original edit-once I\'ve completed my review of the references.  [[User:Legitimus]] appears to have acted partly on the basis of the difficulty of accessing my references for free online and that doesn\'t seem entirely consistent with [[WP:V|Wikipedia verifiability]] guidelines.  But will wait until I thoroughly review both references before I make further edits.[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 04:34, 29 November 2017 (UTC)\\n:::::::I hope you propose the text here first before making the changes to this article or the Child sexual abuse article. Also, per [[WP:Lead]], keep in mind that the lead is meant for summarizing the article. It is not meant for significant detail not first covered lower in the article. \\n\\n:::::::On a side note: There is no need to [[WP:Ping]] me to either article since they are both on my watchlist. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 04:38, 29 November 2017 (UTC)\\n\\n:::::::: Flyer22, I am bending over backwards to act in good faith here.  But there is no precedent either on this talk page or Wikipedia in general to back up your request that I first seek approval of my edits on this talk page.  That is a highly unusual request.  I will do everything I can to act in good faith here but will not be seeking preapproval of my edits on this talk page unless you can point me to a clear Wikipedia policy obligating me to do so.  I also don\'t understand your comment re the lead.  I didn\'t edit the lead in either article.  Willing to act in good faith but you are asking more than is reasonable.[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 05:08, 29 November 2017 (UTC)\\n\\n:::::::::It is not a highly unusual request. Do read [[WP:Consensus]] and [[WP:Bold, revert, discuss]]. Your edit was contested. It has now been contested by two editors. The information you are wanting to add is contentious. Legitimus has already identified problems with what you added. Any further text on this matter should have consensus on this talk page first. Or do you want to get it wrong again and end up being reverted again on the matter? [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 06:10, 29 November 2017 (UTC)\\n\\n:::::::::: Both Wikipedia links you mention say that one should attempt to address all editors\' concerns.  But neither you nor Legitimus have explained what your concerns are.  You don\'t like what I said but haven\'t explained why.  At most Legitimus said he/she couldn\'t access the references I provided for free online.  I already explained per [[WP:V|Wikipedia verifiability]] that isn\'t a valid reason for rejecting my edits.  By no means am I admitting to getting it wrong.  Quite the contrary:  I am 99% percent sure I got it right.  But I am currently doing the legwork to be sure that I am 100% percent sure before reinstating any edits.  You say you are contesting my edits.  What exactly is your problem with my edits?  If you won\'t say how can we hope to achieve consensus?[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 06:41, 29 November 2017 (UTC)\\n\\n:::::::::::Legitmus did explain. His \\"20:42, 27 November 2017 (UTC)\\" post is one example. And I agree with him. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 06:49, 29 November 2017 (UTC)\\n\\n:::::::::::: Legitimus reverted first and didn\'t explain until much, much later.  Not the right order to do things in.  Nevertheless it is definitely my intention in good faith to incorporate Legitimus\' comments of 11/27 20:42 into any new edits I make to this article.  I certainly have an obligation to do that and I believe I already responded in a positive way to Legitimus comments of 11/27 20:42.  I would ask you, with respect, to assume until proven otherwise I will address Legitimus\' concerns in good faith when I edit this article again.  I have been a Wikipedian for 12 years and have some familiarity with how things are done around here.  When I edit this article again, do you personally have any concerns you want me to address beyond Legitimus\' comments of 11/27 20:42?[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 07:15, 29 November 2017 (UTC)\\n:::::::::::: Legitimus\' comments of 11/27 20:42 referenced \\"rape by the son\\".  But so did my original edit to this article.  It is an extremely uncomfortable topic.  But I\'m not clear where you think I \\"got it wrong\\"?[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 07:52, 29 November 2017 (UTC)\\n:To lay out my concerns in detail, the first sentence of the edit was: \\"While incest between adults and children is generally seen as child abuse, there are cases where mother-son incest is initiated by an adolescent son who is the abuser.\\"\\n:That way that is stated is what I see as problematic.  As I said above, the word choice fixates on the matter of it being incest between an adult and a child, missing or failing to emphasize the fact that this is a actually referring to the violent rape of an adult by an nearly-adult.  It comes down to the point you\'re trying to make.  Parent-child incest usually involves non-violent psychological manipulation and an unequal power dynamic of parent and child.  In this way, these two types of incident are completely different, with the only similarity being consanguinity.  As an alternative, how about the following:\\n:\\"While incest between adults and children generally involves the adult as the perpetrator of abuse, there are rare instances of sons sexually assaulting their mothers.  These sons are typically mid to late adolescent and unlike parent-initiated incest, the incidents involve some kind of physical force.  Although the mothers may be accused of being seductive with their sons and inviting the sexual contact, despite strong contrary evidence. Such accusations can parallel other forms of rape, where due to [[victim blaming]] a woman is accused of somehow being at fault for the rape. In some cases mother-son incest is best classified as [[acquaintance rape]] of the mother by the adolescent son.\\"\\n:[[User:Legitimus|Legitimus]] ([[User talk:Legitimus|talk]]) 13:48, 29 November 2017 (UTC)\\n::Sorry it has taken me a few days to respond, and thank you for proposing a constructive change to my proposed edit.  As I mentioned a couple of times, with concerns having been raised I wanted to take the time to fully review the originals of both references I cited.  Now that I\'ve done so, I think I\'m mostly fine with the change you propose, although I would make one small modification.  I would say \\"mid adolescent to young adult\\" rather than \\"mid to late adolescent\\" as both Ward and Courtois, in referring to the relatively rare known cases, clearly note some cases involving young adult rather than adolescent sons.  I will give it a few more days to see if others respond and then make the changes you suggest with my \\"young adult\\" modification.[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 19:37, 2 December 2017 (UTC)\\n:::I\'m ok with that, though will also wait for other commentary.[[User:Legitimus|Legitimus]] ([[User talk:Legitimus|talk]]) 02:32, 3 December 2017 (UTC)\\n::::Page updated as discussed above since there didn\'t seem to be any further input.[[User:Dash77|Dash77]] ([[User talk:Dash77|talk]]) 05:20, 7 December 2017 (UTC)\\n:::::[https://en.wikipedia.org/w/index.php?title=Incest&diff=814238587&oldid=814237494 Tweaked]. [[User:Flyer22 Reborn|Flyer22 Reborn]] ([[User talk:Flyer22 Reborn|talk]]) 16:50, 7 December 2017 (UTC)"}], "ns": 1}}}}'], 'comments': [], 'author': '189.75.148.65', 'cosigners': [{'time_stamp': '03:43, 28 April 2016 (UTC)', 'author': 'Saket4353'}, {'time_stamp': '20:31, 29 July 2015 (UTC)', 'author': 'Legitimus'}, {'time_stamp': '12:55, 6 October 2016 (UTC)', 'author': '41.146.63.190'}, {'time_stamp': '14:11, 6 October 2016 (UTC)', 'author': 'Legitimus'}, {'time_stamp': '03:03, 12 September 2015 (UTC)', 'author': 'CircularReason'}, {'time_stamp': '16:07, 4 June 2017 (UTC)', 'author': 'InternetArchiveBot'}, {'time_stamp': '11:13, 26 July 2017 (UTC)', 'author': 'InternetArchiveBot'}, {'time_stamp': '21:38, 4 August 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '22:30, 4 August 2017 (UTC)', 'author': 'Emir of Wikipedia'}, {'time_stamp': '23:04, 4 August 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '14:09, 5 August 2017 (UTC)', 'author': 'Swingoswingo'}, {'time_stamp': '14:14, 5 August 2017 (UTC)', 'author': 'Swingoswingo'}, {'time_stamp': '14:14, 5 August 2017 (UTC)', 'author': 'Swingoswingo'}, {'time_stamp': '14:17, 6 August 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '13:46, 7 August 2017 (UTC)', 'author': 'Emir of Wikipedia'}, {'time_stamp': '16:16, 7 August 2017 (UTC)', 'author': 'Acroterion'}, {'time_stamp': '22:31, 7 August 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '07:18, 18 August 2017 (UTC)', 'author': 'CaliphoShah'}, {'time_stamp': '19:46, 12 November 2017 (UTC)', 'author': 'InternetArchiveBot'}, {'time_stamp': '17:07, 27 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '18:59, 27 November 2017 (UTC)', 'author': 'Legitimus'}, {'time_stamp': '19:57, 27 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '20:42, 27 November 2017 (UTC)', 'author': 'Legitimus'}, {'time_stamp': '07:20, 28 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '03:39, 29 November 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '04:34, 29 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '04:38, 29 November 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '05:08, 29 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '06:10, 29 November 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '06:41, 29 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '06:49, 29 November 2017 (UTC)', 'author': 'Flyer22 Reborn'}, {'time_stamp': '07:15, 29 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '07:52, 29 November 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '13:48, 29 November 2017 (UTC)', 'author': 'Legitimus'}, {'time_stamp': '19:37, 2 December 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '02:32, 3 December 2017 (UTC)', 'author': 'Legitimus'}, {'time_stamp': '05:20, 7 December 2017 (UTC)', 'author': 'Dash77'}, {'time_stamp': '16:50, 7 December 2017 (UTC)', 'author': 'Flyer22 Reborn'}], 'time_stamp': '23:09, 30 March 2015 (UTC)'}], 'subsections': []}]}																																																																				